<a href="https://colab.research.google.com/github/cc4351/teamtracking/blob/master/Gaussian_Fit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import packages 
# --> run this cell if you plan to run any parts other than part1
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
import statistics
import math
from time import time
import os
from collections import defaultdict
import pickle
import plotly.express as px
from sklearn.preprocessing import normalize

# Data Cleaning

In [ ]:
pkl_file = open('/content/drive/My Drive/data/tracks', 'rb')
tracks = pickle.load(pkl_file)
pkl_file.close()

In [ ]:
# tracks[0]['particles'][0]
# load the Gaussian fit info
movieInfo = loadmat('/content/drive/My Drive/data/resnorm_movieInfo.mat')['movieInfo'] #<=replace the path
resnorm = movieInfo['resnorm']

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
sumResNorm = [];
for item in resnorm:
    tmp = item[0]
    tmpRes = []
    for k in tmp:
        tmpRes.append(sum(map(lambda x:x*x,k)))
    sumResNorm.append(np.asarray(tmpRes))
# resnorm[0][0]
sumResNorm = np.asarray(sumResNorm)

In [ ]:
tracks[10]['particles'][167]
# sumRes = 0
# for item in resnorm[0][0][0]:
#     sumRes += item**2

# print(sumRes == sumResNorm[0][0])

{'amp': 0.0003192056361548017,
 'amp-std': 6.152531707428347e-05,
 'frame': 168,
 'frame_index': 65,
 'pValue': 0.005974072962831589,
 'resnorm': 1.2263442712204418e-07,
 'x': 219.2508919358752,
 'x-std': 0.2906614942409488,
 'y': 125.4519002749874,
 'y-std': 0.2906466452634247}

In [ ]:
# insert the data into Python
for tr in tracks.values():
    for pt in tr['particles']:
        frame = pt['frame']-1
        fr_idx = pt['frame_index']-1
        pt['resnorm'] = sumResNorm[frame][fr_idx]         

In [ ]:
# save the updated tracks.npy
while tracks[len(tracks)-1] == {}:
    del tracks[len(tracks)-1]
output = open('/content/drive/My Drive/Colab Notebooks/Molecule/tracks', 'wb')
pickle.dump(tracks, output)
output.close()

# Analysis of Fit

In [ ]:
fAvgs = [] # for mean of resnorm
fStds = [] # for std of resnorm
pLengths = [] # for trackLength
pStds = [] # for std of pValue
pMean = [] # for mean of pValue

for item in tracks.values():
    pLengths.append(item['length'])
    fList = []
    pList = []
    for pt in item['particles']:
        fList.append(pt['resnorm'])
        pList.append(pt['pValue'])
    fAvgs.append(statistics.mean(fList))
    fStds.append(statistics.pstdev(fList))
    pMean.append(statistics.mean(pList))
    pStds.append(statistics.pstdev(pList))
    

In [ ]:
# a graphying function written in plotly.express
def pxGraph(title, xAxis, yAxis, X, Y, cName = None,colormap = None):
    if colormap != None:
        prep = np.column_stack((X, Y, colormap))
        df = pd.DataFrame(prep, columns=[xAxis, yAxis, cName]) 
        fig = px.scatter(data_frame = df, x=xAxis, y=yAxis, color=cName)
    else:
        prep = np.column_stack((X, Y))
        df = pd.DataFrame(prep, columns=[xAxis, yAxis]) 
        fig = px.scatter(data_frame = df, x=xAxis, y=yAxis)
    fig.update_layout(
        autosize=False,
        width=500,height=375,
        title={'text': title,
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
        yaxis_title=yAxis,
        xaxis_title=xAxis,
        font=dict(
            # family="Courier New, monospace",
            size=15,
            color="#7f7f7f"
        )
    )
    return fig

In [ ]:
# # good old matplotlib --> run this to get a small image
# plt.plot(pAvgs, pLengths, 'x')  
# plt.title('trackLength - meanPValue')
# plt.xlabel('mean pValue')
# plt.ylabel('trackLength')

# for better UX
pxGraph('trackLength - meanResnorm', 'mean resnorm', 'trackLength', fAvgs, pLengths).show()

In [ ]:
pxGraph('log trackLength - log meanResnorm', 'log of mean resnorm', 'log of trackLength', 
        np.log(fAvgs), np.log(pLengths), 'pMean',pMean).show()

In [ ]:
pxGraph('log trackLength - log meanResnorm', 'log of mean resnorm', 'log of trackLength', 
        np.log(fAvgs), np.log(pLengths), 'pStd',pStds).show()

In [ ]:
pxGraph("trackLength - stdResnorm", "resnorm std", "trackLength", fStds, pLengths).show()

In [ ]:
pxGraph('log trackLength - log stdResnorm', 'log of resnorm std', 'log of trackLength', 
        np.log(fStds), np.log(pLengths), 'pMean', pMean).show()

In [ ]:
pxGraph('log trackLength - log stdResnorm', 'log of resnorm std', 'log of trackLength', 
        np.log(fStds), np.log(pLengths), 'pStd', pStds).show()

# Random Snippets (for record keeping)

In [ ]:
# strategy 1: data labels
# strategy 2: colormap
logfStds = np.log(fStds)
logLengths = np.log(pLengths)
prep = np.column_stack((logfStds, logLengths, pMean))
df = pd.DataFrame(prep, columns=['logResnorm', 'logLength', 'pMean']) 
fig = px.scatter(df, x="logResnorm", y="logLength", color=None, size_max=60)
fig.show()

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=logfStds,
    y=logLengths,
    mode="markers",
    hovertext=dummy
))

fig.update_layout(title_text="Hover over the points to see the text")

fig.show()